In [127]:
#Author: ML Tlachac, WPI
#For DepreST-CAT, 2021

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

In [171]:
labels = ["phq9", "gad7"]
modelTypelist = ["SVC", "kNN", "RF", "LR", "XG"]
splits = [5,6,7,8,9,10]

for split in splits: 
    
    for week in [2,4,8,16]:

        data = pd.read_csv("featureSet" + str(week) + "weeksDepreST-CAT.csv")
        print(data.shape)

        #binary labels
        d10 = []
        g10 = []
        for i in range(0, data.shape[0]):
            if int(data.phq9[i]) >= split:
                d10.append(1)
            else:
                d10.append(0)
            if int(data.gad7[i]) >= split:
                g10.append(1)
            else:
                g10.append(0)
        data["phq9"] = d10
        data["gad7"] = g10

        print(data.shape)
        
        for label in labels:

            #create lists to populate
            flist = [] 
            mlist = []
            llist = []
            featureList = []
            wlist = []
            slist = []
            f1List = []
            accuracyList = []
            truePosList = []
            trueNegList = []
            falsePosList = []
            falseNegList = []
            predictions = []
            rseed = []

            for r in range(0, 100):

                #train/test split    
                df_train, df_test = train_test_split(data, test_size=0.3, stratify=data[["phq9", "gad7"]], random_state = r)
                trainids = list(df_train["id"])
                testids = list(df_test["id"])
                print(data.shape)
                testdata = data[data['id'].isin(testids)]
                print(testdata.shape)
                traindata = data[data['id'].isin(trainids)]
                print(traindata.shape)

                #limit to features
                testContent = testdata[testdata.columns[1:-2]]
                print(testContent.shape)
                trainContent = traindata[traindata.columns[1:-2]]
                print(trainContent.shape)

                #NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION
                min_max_scaler = preprocessing.MinMaxScaler()  
                np_scaled = min_max_scaler.fit_transform(trainContent)
                featureSubset = pd.DataFrame(np_scaled)
                np_scaled2 =  min_max_scaler.transform(testContent)
                testSubset = pd.DataFrame(np_scaled2)
                print(featureSubset.shape)
                print(testSubset.shape)

                target = list(traindata[label])

                featureDF = []
                testDFs = []

                nFeatureList = list(np.arange(1,5,1))
                for numberOfFeatures in nFeatureList:
                    pca = PCA(n_components=numberOfFeatures)
                    pca = pca.fit(featureSubset)
                    X_pca = pca.transform(featureSubset)
                    pcaDF = pd.DataFrame(X_pca)
                    pcaDF = pcaDF.assign(target = target)
                    featureDF.append(pcaDF)
                    testSubset2 = pca.transform(testSubset)
                    testDFs.append(pd.DataFrame(testSubset2))


                for f in range(0, len(featureDF)):
                    print(f)

                    train_phq9 = featureDF[f]
                    X_test = testDFs[f]

                    # upsampling 
                    #Count 1s and 0s
                    ones = len(train_phq9.loc[train_phq9['target'] == 1])
                    zeros = len(train_phq9.loc[train_phq9['target'] == 0])
                    if ones >= zeros:
                        majority = 1
                        minority = 0
                    else:
                        majority = 0
                        minority = 1


                    # Upsample TrainingSet 
                    train_majority = train_phq9[train_phq9.target==majority]
                    train_minority = train_phq9[train_phq9.target==minority]

                    #print("train_majority ="  + str(len(train_majority)))
                    #print("train_minority ="  + str(len(train_minority)))

                    # Upsample minority class
                    train_minority_upsampled = resample(train_minority, 
                                                     replace=True,     # sample with replacement
                                                     n_samples=len(train_majority),    # to match majority class
                                                     random_state=42) # reproducible results

                    # Combine majority class with upsampled minority class
                    train_phq9 = pd.concat([train_majority, train_minority_upsampled])

                    #seperate features and target
                    y_train = train_phq9["target"]
                    X_train = train_phq9.drop(columns = "target")

                    for modelType in modelTypelist:

                        #add data to lists
                        llist.append(label)
                        featureList.append(f +1)
                        flist.append("PCA")
                        mlist.append(modelType)

                        #chose model type
                        if modelType == "SVC":
                            clf = svm.SVC(random_state=r)
                        elif modelType == "RF":
                            clf = RandomForestClassifier(random_state=r)
                        elif modelType == "kNN":
                            clf = KNeighborsClassifier()
                        elif modelType == "LR":
                            clf = LogisticRegression(random_state=r)
                        elif modelType =="XG":
                            clf = xgb.XGBClassifier(random_state=r)

                        #train model and make predictions
                        clf.fit(X_train, y_train)
                        y_pred = clf.predict(X_test)

                        #evaluate model
                        conf_mat = confusion_matrix(list(testdata[label]), y_pred)
                        TN = conf_mat[0][0]
                        TP = conf_mat[1][1]
                        FP = conf_mat[0][1]
                        FN = conf_mat[1][0]
                        precision = TP/(TP+FP)
                        sensitivity = TP/(TP+FN)
                        f1 = (2*precision*sensitivity)/(precision + sensitivity)
                        accuracy = (TP+TN)/(TN+TP+FP+FN)

                        #populate lists with results
                        f1List.append(f1)
                        accuracyList.append(accuracy)
                        truePosList.append(TP)
                        trueNegList.append(TN)
                        falsePosList.append(FP)
                        falseNegList.append(FN)
                        predictions.append(y_pred)
                        rseed.append(r)
                        slist.append(split)
                        wlist.append(week)

            resultsDF = pd.DataFrame()
            resultsDF["week"] = wlist
            resultsDF["label"] = llist
            resultsDF["split"] = slist
            resultsDF["Engineering"] = flist
            resultsDF["model"] = mlist
            resultsDF["nFeatures"] = featureList
            resultsDF["F1"] = f1List
            resultsDF["Accuracy"] = accuracyList
            resultsDF["truePos"] = truePosList
            resultsDF["trueNeg"] = trueNegList
            resultsDF["falsePos"] = falsePosList
            resultsDF["falseNeg"] = falseNegList
            resultsDF["predictions"] = predictions
            resultsDF["randomSeed"] = rseed

            resultsDF.to_csv("results/resultsCAT" + str(week) + "week" + label + "split" + str(split) + ".csv")

(369, 571)
(369, 571)
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)

1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258

1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 571)
(111, 571)
(258, 571)
(111, 568)
(258, 568)
(258, 568)
(111, 568)
0
1
2
3
(369, 599)
(369, 599)
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111

1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258

1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258, 596)
(111, 596)
0
1
2
3
(369, 599)
(111, 599)
(258, 599)
(111, 596)
(258, 596)
(258

...

1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258

0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(258, 652)
(111, 652)
0
1
2
3
(369, 655)
(111, 655)
(258, 655)
(111, 652)
(258, 652)
(2

1
2
3
(369, 767)
(369, 767)
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111

0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(2

0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3
(369, 767)
(111, 767)
(258, 767)
(111, 764)
(258, 764)
(258, 764)
(111, 764)
0
1
2
3


In [176]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split5.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.68, 0.66, 0.67, 0.67
LR F1: 0.7, 0.56, 0.58, 0.63
SVC F1: 0.58, 0.58, 0.59, 0.63
RF F1: 0.76, 0.77, 0.78, 0.79
XG F1: 0.75, 0.77, 0.78, 0.79
WEEK:4
kNN F1: 0.68, 0.66, 0.66, 0.64
LR F1: 0.7, 0.59, 0.61, 0.64
SVC F1: 0.52, 0.56, 0.59, 0.62
RF F1: 0.77, 0.78, 0.79, 0.78
XG F1: 0.76, 0.78, 0.78, 0.78
WEEK:8
kNN F1: 0.68, 0.67, 0.68, 0.68
LR F1: 0.73, 0.63, 0.63, 0.64
SVC F1: 0.53, 0.56, 0.59, 0.63
RF F1: 0.77, 0.77, 0.79, 0.79
XG F1: 0.76, 0.77, 0.78, 0.79
WEEK:16
kNN F1: 0.7, 0.69, 0.67, 0.67
LR F1: 0.72, 0.63, 0.64, 0.64
SVC F1: 0.5, 0.54, 0.57, 0.58
RF F1: 0.78, 0.78, 0.79, 0.8
XG F1: 0.78, 0.79, 0.79, 0.8
______________gad7_______________
WEEK:2
kNN F1: 0.63, 0.62, 0.6, 0.6
LR F1: 0.61, 0.54, 0.59, 0.61
SVC F1: 0.56, 0.54, 0.58, 0.58
RF F1: 0.69, 0.7, 0.71, 0.71
XG F1: 0.68, 0.7, 0.71, 0.72
WEEK:4
kNN F1: 0.63, 0.63, 0.64, 0.64
LR F1: 0.62, 0.57, 0.62, 0.63
SVC F1: 0.59, 0.6, 0.62, 0.62
RF F1: 0.7, 0.71, 0.72, 0.72
XG F1: 0.69, 0.71, 

In [177]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split6.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.65, 0.65, 0.64, 0.65
LR F1: 0.71, 0.51, 0.54, 0.59
SVC F1: 0.61, 0.58, 0.6, 0.62
RF F1: 0.72, 0.73, 0.74, 0.75
XG F1: 0.7, 0.73, 0.73, 0.74
WEEK:4
kNN F1: 0.65, 0.64, 0.63, 0.63
LR F1: 0.7, 0.53, 0.55, 0.6
SVC F1: 0.61, 0.59, 0.6, 0.63
RF F1: 0.73, 0.73, 0.75, 0.75
XG F1: 0.72, 0.74, 0.74, 0.75
WEEK:8
kNN F1: 0.65, 0.62, 0.63, 0.65
LR F1: 0.71, 0.58, 0.6, 0.61
SVC F1: 0.59, 0.58, 0.59, 0.6
RF F1: 0.73, 0.71, 0.73, 0.74
XG F1: 0.72, 0.72, 0.73, 0.74
WEEK:16
kNN F1: 0.66, 0.63, 0.62, 0.62
LR F1: 0.68, 0.57, 0.61, 0.62
SVC F1: 0.5, 0.53, 0.55, 0.55
RF F1: 0.74, 0.73, 0.75, 0.75
XG F1: 0.73, 0.74, 0.75, 0.74
______________gad7_______________
WEEK:2
kNN F1: 0.58, 0.6, 0.57, 0.57
LR F1: 0.54, 0.5, 0.54, 0.55
SVC F1: 0.47, 0.49, 0.53, 0.54
RF F1: 0.64, 0.65, 0.66, 0.67
XG F1: 0.63, 0.65, 0.66, 0.67
WEEK:4
kNN F1: 0.59, 0.58, 0.59, 0.59
LR F1: 0.56, 0.5, 0.54, 0.56
SVC F1: 0.48, 0.5, 0.52, 0.53
RF F1: 0.65, 0.65, 0.67, 0.68
XG F1: 0.64, 0.65, 

In [178]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split7.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.62, 0.61, 0.6, 0.6
LR F1: 0.63, 0.58, 0.57, 0.59
SVC F1: 0.58, 0.6, 0.6, 0.61
RF F1: 0.69, 0.69, 0.71, 0.71
XG F1: 0.68, 0.69, 0.7, 0.71
WEEK:4
kNN F1: 0.63, 0.62, 0.61, 0.59
LR F1: 0.63, 0.6, 0.59, 0.59
SVC F1: 0.54, 0.62, 0.61, 0.61
RF F1: 0.7, 0.71, 0.72, 0.72
XG F1: 0.69, 0.71, 0.72, 0.72
WEEK:8
kNN F1: 0.61, 0.6, 0.62, 0.62
LR F1: 0.67, 0.64, 0.62, 0.61
SVC F1: 0.6, 0.63, 0.63, 0.63
RF F1: 0.7, 0.68, 0.71, 0.71
XG F1: 0.69, 0.69, 0.7, 0.71
WEEK:16
kNN F1: 0.61, 0.59, 0.61, 0.61
LR F1: 0.66, 0.63, 0.62, 0.61
SVC F1: 0.56, 0.63, 0.61, 0.61
RF F1: 0.7, 0.69, 0.71, 0.71
XG F1: 0.69, 0.7, 0.71, 0.71
______________gad7_______________
WEEK:2
kNN F1: 0.56, 0.58, 0.57, 0.58
LR F1: 0.6, 0.47, 0.54, 0.55
SVC F1: 0.5, 0.48, 0.52, 0.53
RF F1: 0.61, 0.61, 0.62, 0.63
XG F1: 0.6, 0.61, 0.62, 0.63
WEEK:4
kNN F1: 0.56, 0.57, 0.58, 0.58
LR F1: 0.61, 0.49, 0.54, 0.56
SVC F1: 0.48, 0.47, 0.49, 0.51
RF F1: 0.61, 0.63, 0.64, 0.64
XG F1: 0.6, 0.63, 0.64,

In [179]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split8.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.59, 0.59, 0.58, 0.58
LR F1: 0.65, 0.55, 0.55, 0.57
SVC F1: 0.58, 0.6, 0.57, 0.59
RF F1: 0.65, 0.66, 0.67, 0.67
XG F1: 0.64, 0.66, 0.67, 0.68
WEEK:4
kNN F1: 0.59, 0.57, 0.58, 0.58
LR F1: 0.65, 0.57, 0.55, 0.57
SVC F1: 0.58, 0.58, 0.57, 0.58
RF F1: 0.66, 0.65, 0.67, 0.68
XG F1: 0.65, 0.66, 0.67, 0.68
WEEK:8
kNN F1: 0.59, 0.56, 0.59, 0.6
LR F1: 0.67, 0.62, 0.61, 0.61
SVC F1: 0.62, 0.61, 0.58, 0.6
RF F1: 0.66, 0.63, 0.65, 0.67
XG F1: 0.64, 0.64, 0.65, 0.67
WEEK:16
kNN F1: 0.59, 0.58, 0.57, 0.57
LR F1: 0.67, 0.63, 0.59, 0.58
SVC F1: 0.53, 0.56, 0.53, 0.54
RF F1: 0.67, 0.65, 0.66, 0.67
XG F1: 0.66, 0.66, 0.66, 0.66
______________gad7_______________
WEEK:2
kNN F1: 0.53, 0.52, 0.51, 0.52
LR F1: 0.58, 0.46, 0.49, 0.49
SVC F1: 0.49, 0.46, 0.49, 0.49
RF F1: 0.57, 0.56, 0.56, 0.56
XG F1: 0.56, 0.56, 0.57, 0.57
WEEK:4
kNN F1: 0.51, 0.52, 0.54, 0.52
LR F1: 0.57, 0.47, 0.49, 0.49
SVC F1: 0.44, 0.45, 0.47, 0.48
RF F1: 0.57, 0.56, 0.57, 0.57
XG F1: 0.5

In [180]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split9.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.59, 0.59, 0.57, 0.56
LR F1: 0.58, 0.49, 0.54, 0.58
SVC F1: 0.55, 0.58, 0.6, 0.6
RF F1: 0.64, 0.64, 0.65, 0.65
XG F1: 0.63, 0.65, 0.65, 0.66
WEEK:4
kNN F1: 0.59, 0.58, 0.57, 0.58
LR F1: 0.57, 0.5, 0.53, 0.56
SVC F1: 0.52, 0.58, 0.59, 0.6
RF F1: 0.64, 0.65, 0.67, 0.67
XG F1: 0.63, 0.65, 0.66, 0.66
WEEK:8
kNN F1: 0.58, 0.56, 0.57, 0.59
LR F1: 0.59, 0.53, 0.55, 0.57
SVC F1: 0.54, 0.55, 0.55, 0.59
RF F1: 0.64, 0.63, 0.65, 0.66
XG F1: 0.63, 0.64, 0.65, 0.65
WEEK:16
kNN F1: 0.58, 0.56, 0.55, 0.55
LR F1: 0.59, 0.55, 0.53, 0.55
SVC F1: 0.52, 0.55, 0.53, 0.55
RF F1: 0.64, 0.63, 0.63, 0.64
XG F1: 0.64, 0.64, 0.64, 0.64
______________gad7_______________
WEEK:2
kNN F1: 0.43, 0.44, 0.43, 0.43
LR F1: 0.49, 0.43, 0.45, 0.47
SVC F1: nan, 0.43, 0.45, 0.45
RF F1: 0.39, 0.38, 0.36, 0.37
XG F1: 0.39, 0.38, 0.38, 0.38
WEEK:4
kNN F1: 0.44, 0.44, 0.46, 0.45
LR F1: 0.46, 0.43, 0.45, 0.46
SVC F1: 0.37, 0.43, 0.44, 0.44
RF F1: 0.39, 0.39, 0.39, 0.39
XG F1: 0.39,

In [181]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2', "4", "8", "16"]:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split10.csv")
        for model in ["kNN", "LR", "SVC", "RF", "XG"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1]
            d2 = d[d.nFeatures == 2]
            d3 = d[d.nFeatures == 3]
            d4 = d[d.nFeatures == 4]
            print(model + " " +  str(score) + ": " + str(round(sum(d1[score])/100,2)) + ", " + str(round(sum(d2[score])/100,2)) + ", " + str(round(sum(d3[score])/100,2)) + ", " + str(round(sum(d4[score])/100,2)))

______________phq9_______________
WEEK:2
kNN F1: 0.56, 0.54, 0.53, 0.53
LR F1: 0.56, 0.49, 0.5, 0.52
SVC F1: 0.52, 0.51, 0.52, 0.53
RF F1: 0.6, 0.59, 0.59, 0.6
XG F1: 0.59, 0.59, 0.59, 0.61
WEEK:4
kNN F1: 0.55, 0.53, 0.54, 0.53
LR F1: 0.53, 0.5, 0.51, 0.52
SVC F1: 0.44, 0.52, 0.52, 0.54
RF F1: 0.6, 0.59, 0.6, 0.6
XG F1: 0.59, 0.59, 0.6, 0.61
WEEK:8
kNN F1: 0.54, 0.53, 0.54, 0.56
LR F1: 0.57, 0.54, 0.53, 0.54
SVC F1: 0.49, 0.51, 0.49, 0.52
RF F1: 0.6, 0.58, 0.61, 0.61
XG F1: 0.59, 0.59, 0.6, 0.61
WEEK:16
kNN F1: 0.56, 0.54, 0.52, 0.54
LR F1: 0.58, 0.55, 0.52, 0.53
SVC F1: nan, 0.49, 0.45, 0.47
RF F1: 0.61, 0.6, 0.61, 0.61
XG F1: 0.61, 0.6, 0.6, 0.61
______________gad7_______________
WEEK:2
kNN F1: 0.43, 0.42, 0.41, 0.42
LR F1: 0.5, 0.41, 0.42, 0.45
SVC F1: 0.42, 0.41, 0.44, 0.45
RF F1: 0.37, 0.37, 0.34, 0.36
XG F1: 0.38, 0.36, 0.34, 0.36
WEEK:4
kNN F1: 0.43, 0.43, 0.43, 0.44
LR F1: 0.47, 0.41, 0.43, 0.44
SVC F1: nan, 0.41, 0.43, 0.43
RF F1: 0.37, 0.36, 0.36, 0.35
XG F1: 0.38, 0.37, 0.37